In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# 데이터 로드
from PIL import Image
import numpy as np
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        if img is not None:
            images.append(np.array(img))
    return np.array(images)

In [ ]:
folder_path = '/content/drive/MyDrive/colab_data/COVID_19_XRAY/train'
train_images = load_images_from_folder(folder_path)


In [ ]:
import pandas as pd

labels_df = pd.read_csv('/content/drive/MyDrive/colab_data/COVID_19_XRAY/labels.csv')
labels = labels_df['label']

for i, label in enumerate(labels):
    if label == 'covid':
        labels[i] = 0
    elif label == 'normal':
        labels[i] = 1
    else:
        print(f"Unexpected label: {label}")

train_labels = labels
train_labels

0       0
1       0
2       1
3       1
4       1
       ..
1995    0
1996    1
1997    0
1998    1
1999    1
Name: label, Length: 2000, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

train_images1, val_images1, train_labels1, val_labels1 = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

In [ ]:
train_images1 = train_images1.reshape((1800, 299, 299, 1))
train_images1 = train_images1 / 255.0

val_images1 = val_images1.reshape((200, 299, 299, 1))
val_labels1 = val_labels1 / 255.0

In [ ]:
import tensorflow as tf

# GPU 메모리 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


In [ ]:
train_labels1 = train_labels1.astype(int)
val_labels1 = val_labels1.astype(int)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images1, train_labels1, epochs=50, batch_size=128,
                    validation_data=(val_images1, val_labels1))

Epoch 1/50
15/15 [==============================] - 8s 312ms/step - loss: 0.7677 - accuracy: 0.4978 - val_loss: 2.8123 - val_accuracy: 0.0000e+00
Epoch 2/50
15/15 [==============================] - 4s 295ms/step - loss: 0.6932 - accuracy: 0.5117 - val_loss: 2.1488 - val_accuracy: 0.0000e+00
Epoch 3/50
15/15 [==============================] - 5s 306ms/step - loss: 0.6956 - accuracy: 0.4989 - val_loss: 0.5660 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 4s 296ms/step - loss: 0.6935 - accuracy: 0.5044 - val_loss: 0.6536 - val_accuracy: 0.9750
Epoch 5/50
15/15 [==============================] - 4s 291ms/step - loss: 0.6931 - accuracy: 0.5072 - val_loss: 0.7015 - val_accuracy: 0.2200
Epoch 6/50
15/15 [==============================] - 5s 303ms/step - loss: 0.6931 - accuracy: 0.5056 - val_loss: 1.2524 - val_accuracy: 0.0250
Epoch 7/50
15/15 [==============================] - 4s 295ms/step - loss: 0.6931 - accuracy: 0.5056 - val_loss: 0.5163 - val_accuracy: 0.875

In [ ]:
test_loss, test_acc = model.evaluate(val_images1, val_labels1)
print('Test accuracy:', test_acc)

7/7 [==============================] - 0s 31ms/step - loss: 471.3141 - accuracy: 0.6850
Test accuracy: 0.6850000023841858


In [ ]:
folder_path = '/content/drive/MyDrive/colab_data/COVID_19_XRAY/test'
test_images = load_images_from_folder(folder_path)

test_images

array([[[  0,   0,   0, ...,   0,   0,   2],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [ 18,  15,  12, ...,   0,   0,   0],
        [  7,   8,  11, ...,   0,   0,   0],
        [  6,  10,  13, ...,   0,   0,   0]],

       [[ 28,  27,  24, ...,  15,  16,  15],
        [ 31,  30,  27, ...,  18,  19,  17],
        [ 30,  30,  28, ...,  18,  18,  18],
        ...,
        [221, 245, 246, ..., 233, 231, 211],
        [170, 190, 191, ..., 182, 180, 164],
        [ 12,  12,  11, ...,  11,  12,  11]],

       [[ 62,  62,  62, ...,  73,  77,  85],
        [ 10,  10,  10, ...,  18,  23,  34],
        [  0,   0,   0, ...,   9,  15,  26],
        ...,
        [134,  79,  44, ...,  15,  16,  18],
        [139,  86,  52, ...,  24,  25,  26],
        [166, 125, 100, ...,  76,  77,  79]],

       ...,

       [[ 10,   3,   1, ...,   3,   4,   5],
        [  9,   2,   1, ...,   1,   2,   2],
        [  9,   2,   1, ...,   1,   1,   0

In [ ]:
test_images = test_images.reshape((400, 299, 299, 1))
test_images = test_images / 255.0

In [ ]:
import csv

def get_predicted_labels(model, test_images):    
    # 모델을 사용하여 예측
    predictions = model.predict(test_images)
    predicted_labels = (np.array(predictions) > 0.5).astype('int32')
    return predicted_labels

def save_labels_to_csv(labels, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'label'])
        for i, label in enumerate(labels):
            writer.writerow([i, label])

In [ ]:
predict_labels = get_predicted_labels(model, test_images)
save_labels_to_csv(predict_labels, '/content/drive/MyDrive/colab_data/COVID_19_XRAY/predict_labels')

13/13 [==============================] - 0s 29ms/step
